In [2]:
## On Not being able to use python for Despina's. She said in L10 that the only thing she wants to see is that we've done it and that it wont be marked.
## Only assessed on whether it works or not

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import norm
from datetime import datetime, time 

## Importing hand-made modules
import sys
scraper_modules_path = 'C:/Users/ABour/Desktop/Energy Systems & Data Analytics/Modules/Energy Analysis - Aidan/Smart Meters/custom_modules'
if scraper_modules_path not in sys.path:
    sys.path.append(scraper_modules_path)

import pre_processing as pp
import feature_gen as fg

<br>

### Pre-processing & EDA

In [2]:
df_smart_meters = (pd.read_csv('data/smart_meters.csv', parse_dates=['DateTime'])
                   .drop(columns=['Acorn', 'Acorn_grouped'])
                   .rename({'LCLid':'id', 'KWH/hh (per half hour) ':'kWh', 'DateTime':'datetime'}, axis=1)
                  )
df_smart_meters

,id,stdorToU,datetime,kWh
0,MAC000002,Std,2012-10-12 00:30:00,0
1,MAC000002,Std,2012-10-12 01:00:00,0
2,MAC000002,Std,2012-10-12 01:30:00,0
3,MAC000002,Std,2012-10-12 02:00:00,0
4,MAC000002,Std,2012-10-12 02:30:00,0
5,MAC000002,Std,2012-10-12 03:00:00,0
6,MAC000002,Std,2012-10-12 03:30:00,0
7,MAC000002,Std,2012-10-12 04:00:00,0
8,MAC000002,Std,2012-10-12 04:30:00,0
9,MAC000002,Std,2012-10-12 05:00:00,0


In [8]:
## Drop ToU tariff households, split remaining std tariff households into 4 and pickle each of them
all_ids = df_smart_meters.id.unique()
ToU_ids = df_smart_meters[df_smart_meters.stdorToU=='ToU'].id.unique()
std_ids = list(set(all_ids) - set(ToU_ids))

In [23]:
## Number of households who use each tariff
num_ToU = len(list(ToU_ids))
num_non_ToU = len(list(std_ids))
print(f'Number of households who use ToU tariff: {num_ToU}')
print(f'Number of households with just std tariff: {num_non_ToU}')

Number of households who use ToU tariff: 1123
Number of households with just std tariff: 4443


In [48]:
## Defining household IDs for batch runs
quarter_non_toU = int(num_non_ToU/4)
batch_1_idxs = [0, quarter_non_toU]
batch_2_idxs = [batch_1_idxs[1], 2*quarter_non_toU]
batch_3_idxs = [batch_2_idxs[1], 3*quarter_non_toU]
batch_4_idxs = [batch_3_idxs[1], num_non_ToU]

batch_1_ids = std_ids[batch_1_idxs[0]:batch_1_idxs[1]]
batch_2_ids = std_ids[batch_2_idxs[0]:batch_2_idxs[1]]
batch_3_ids = std_ids[batch_3_idxs[0]:batch_3_idxs[1]]
batch_4_ids = std_ids[batch_4_idxs[0]:batch_4_idxs[1]]
batch_5_ids = list(ToU_ids)

## Sanity check that no IDs have been lost, should always output 0
batch_sets = set(batch_1_ids+batch_2_ids+batch_3_ids+batch_4_ids+batch_5_ids)
ids_set = set(all_ids)
num_missing_ids = len(ids_set - batch_sets) + len(batch_sets - ids_set)
print(f'Number of missing IDs for batch runs: {num_missing_ids}')

Number of missing IDs for batch runs: 0


In [55]:
## Creating and pickling dataframes for each batch
df_batch_1 = df_smart_meters[df_smart_meters.id.isin(batch_1_ids)]
df_batch_2 = df_smart_meters[df_smart_meters.id.isin(batch_2_ids)]
df_batch_3 = df_smart_meters[df_smart_meters.id.isin(batch_3_ids)]
df_batch_4 = df_smart_meters[df_smart_meters.id.isin(batch_4_ids)]
df_batch_5 = df_smart_meters[df_smart_meters.id.isin(batch_5_ids)]

df_batch_1.to_pickle('data/batch_1.pkl')
df_batch_2.to_pickle('data/batch_2.pkl')
df_batch_3.to_pickle('data/batch_3.pkl')
df_batch_4.to_pickle('data/batch_4.pkl')
df_batch_5.to_pickle('data/batch_5.pkl')